IMF에서 제공하는 국가별 GDP를 구하세요. (https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29)

국가별 GDP를 확인할 수 있는 테이블을 만드세요.

해당 테이블에는 GDP가 높은 국가들이 먼저 나와야 합니다.

GDP의 단위는 1B USD이어야 하고 소수점 2자리까지만 표시해 주세요.

IMF에서 매년 2회 이 자료를 제공하기 때문에 정보가 갱신되더라도 해당 코드를 재사용해서 정보를 얻을 수 있어야 합니다.



GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.

각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다.

# Extract

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import requests

# Extract

# url에서 원본 HTML 로드, bs4 객체회
print("Start Extraction")
response = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29')
if response.status_code != 200:
    print(response.status_code)
    raise Exception("Failed to load page")
else:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

# 
table = soup.find('table', {'class':'wikitable'})
rows = table.find_all('tr')

print("End Extraction")


Start Extraction
End Extraction


In [2]:
print("Start Transformation")

countries = []
gdps = []

for i, row in enumerate(rows):
    # 테이블 메타데이터
    if i in [0, 1, 2]:
        continue
    # 각 행의 열 정보들 리스트화
    cols = row.find_all('td')
    for j, col in enumerate(cols):
        # 국가 정보
        if j == 0:
            # print(col.get_text(strip=True))
            countries.append(col.get_text(strip=True))
        # IMF의 GDP정보.
        if j == 1:
            content = col.get_text(strip=True)
            # IMF 미가입 국가
            if content == '—':
                gdps.append(-1)
            else:
                gdps.append(float(content.replace(',', '')))

Start Transformation


In [3]:
# Pandas DataFrame화
df = pd.DataFrame({'Country':countries, 'GDP':gdps})

# IMF 미가입 국가의 데이터값 Not a Number로 변경
df['GDP'] = df['GDP'].replace(-1, np.nan)
# GDP 값 1B USD 단위로 변경
df['GDP'] = (df['GDP']/1000).round(2)

# GDP 높은 순으로 정렬
df = df.sort_values('GDP', ascending=False)

# 생성한 DF .json으로 저장
df.to_json("Countries_by_GDP.json", orient='records')

In [4]:
df

,Country,GDP
0,United States,30507.22
1,China,19231.71
2,Germany,4744.80
3,India,4187.02
4,Japan,4186.43
...,...,...
207,American Samoa,NaN
210,Saint Martin,NaN
213,Anguilla,NaN
214,Cook Islands,NaN


In [5]:
# GDP 100B USD 이상의 국가 출력
df_100B = df[df['GDP'] >= 100]
df_100B

,Country,GDP
0,United States,30507.22
1,China,19231.71
2,Germany,4744.80
3,India,4187.02
4,Japan,4186.43
...,...,...
69,Bulgaria,117.01
70,Angola,113.34
71,Venezuela,108.51
72,Oman,104.35


In [6]:
# Region별
african = 'https://en.wikipedia.org/wiki/List_of_African_countries_by_GDP_(nominal)'
arab = 'https://en.wikipedia.org/wiki/List_of_Arab_League_countries_by_GDP_(nominal)'
asia_pac = 'https://en.wikipedia.org/wiki/List_of_countries_in_Asia-Pacific_by_GDP_(nominal)'
latin = 'https://en.wikipedia.org/wiki/List_of_Latin_American_and_Caribbean_countries_by_GDP_(nominal)'
na = 'https://en.wikipedia.org/wiki/List_of_North_American_countries_by_GDP_(nominal)'
oceanian = 'https://en.wikipedia.org/wiki/List_of_Oceanian_countries_by_GDP'
europe = 'https://en.wikipedia.org/wiki/List_of_sovereign_states_in_Europe_by_GDP_(nominal)'

In [7]:
response = requests.get(na)
if response.status_code != 200:
    print(response.status_code)
    raise Exception("Failed to load page")
else:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

table = soup.find('table', {'class':'wikitable'})
rows = table.find_all('tr')

df = pd.read_json("Countries_by_GDP.json")
countries = df['Country'].to_list()

region_countries = []

for row in rows:
    cols = row.find_all('a')
    for col in cols:
        country = col.get_text().strip()

        if country in countries:
            region_countries.append(country)
        
print(region_countries)

['United States', 'Canada', 'Mexico', 'Dominican Republic', 'Guatemala', 'Cuba', 'Costa Rica', 'Panama', 'Honduras', 'El Salvador', 'Haiti', 'Trinidad and Tobago', 'Jamaica', 'Nicaragua', 'Barbados', 'Belize', 'Saint Lucia', 'Antigua and Barbuda', 'Grenada', 'Saint Vincent and the Grenadines', 'Saint Kitts and Nevis', 'Dominica']


In [8]:
def get_country_from_region(region : str):

    # Region별 url
    urls = {
        'african' : 'https://en.wikipedia.org/wiki/List_of_African_countries_by_GDP_(nominal)',
        'arab' : 'https://en.wikipedia.org/wiki/List_of_Arab_League_countries_by_GDP_(nominal)',
        'ap' : 'https://en.wikipedia.org/wiki/List_of_countries_in_Asia-Pacific_by_GDP_(nominal)',
        'latin' : 'https://en.wikipedia.org/wiki/List_of_Latin_American_and_Caribbean_countries_by_GDP_(nominal)',
        'na' : 'https://en.wikipedia.org/wiki/List_of_North_American_countries_by_GDP_(nominal)',
        'oceanian' : 'https://en.wikipedia.org/wiki/List_of_Oceanian_countries_by_GDP',
        'europe' : 'https://en.wikipedia.org/wiki/List_of_sovereign_states_in_Europe_by_GDP_(nominal)',
    }

    if(region not in urls.keys()):
        print("Region not avaiable.")
        print("Select from ['african', 'arab', 'ap', 'latin', 'na', 'oceanian', 'europe']")
        return

    response = requests.get(urls[region])
    if response.status_code != 200:
        print(response.status_code)
        raise Exception("Failed to load page")
    else:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

    table = soup.find('table', {'class':'wikitable'})
    rows = table.find_all('tr')

    df = pd.read_json("Countries_by_GDP.json")
    countries = df['Country'].to_list()

    region_countries = []

    for row in rows:
        cols = row.find_all('a')
        for col in cols:
            country = col.get_text().strip()

            if country in countries:
                region_countries.append(country)

    return region_countries


countries = []
gdps = []
regions_str = ['african', 'arab', 'ap', 'latin', 'na' ,'oceanian', 'europe']
african = get_country_from_region('african')
arab = get_country_from_region('arab')
ap = get_country_from_region('ap')
latin = get_country_from_region('latin')
na = get_country_from_region('na')
oceanian = get_country_from_region("oceanian")
europe = get_country_from_region('europe')
regions = [african, arab, ap, latin, na, oceanian, europe]
isRegion = [[] for  i in range(len(regions))]

response = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29')
if response.status_code != 200:
    print(response.status_code)
    raise Exception("Failed to load page")
else:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

table = soup.find('table', {'class':'wikitable'})
rows = table.find_all('tr')

for i, row in enumerate(rows):
    # 테이블 메타데이터
    if i in [0, 1, 2]:
        continue
    # 각 행의 열 정보들 리스트화
    cols = row.find_all('td')
    for j, col in enumerate(cols):
        # 국가 정보
        if j == 0:
            # print(col.get_text(strip=True))
            country = col.get_text(strip=True)
            countries.append(country)
            for k, region in enumerate(regions):
                if(country in region):
                    isRegion[k].append(1)
                else:
                    isRegion[k].append(0)

        # IMF의 GDP정보.
        if j == 1:
            content = col.get_text(strip=True)
            # IMF 미가입 국가
            if content == '—':
                gdps.append(-1)
            else:
                # GDP 정보 추출 후 USD billoin 단위로 변환
                gdp = float(content.replace(',', ''))
                gdp /= 1000
                gdp = round(gdp, 2)
                gdps.append(gdp)
    
# Pandas DataFrame화
df = pd.DataFrame({'Country':countries, 'GDP_USD_billion':gdps})
# 각 Region 속성 추가
for i, region in enumerate(regions_str):
    df[region] = isRegion[i]

# IMF 미가입 국가의 데이터값 Not a Number로 변경
df['GDP_USD_billion'] = df['GDP_USD_billion'].replace(-1, np.nan)

# GDP 높은 순으로 정렬
df = df.sort_values('GDP_USD_billion', ascending=False)

print("End Transformation")

df.head(10)

End Transformation


,Country,GDP_USD_billion,african,arab,ap,latin,na,oceanian,europe
0,United States,30507.22,0,0,0,0,1,0,0
1,China,19231.71,0,0,1,0,0,0,0
2,Germany,4744.80,0,0,0,0,0,0,1
3,India,4187.02,0,0,1,0,0,0,0
4,Japan,4186.43,0,0,1,0,0,0,0
5,United Kingdom,3839.18,0,0,0,0,0,0,1
6,France,3211.29,0,0,0,0,0,0,1
7,Italy,2422.86,0,0,0,0,0,0,1
8,Canada,2225.34,0,0,0,0,1,0,0
9,Brazil,2125.96,0,0,0,1,0,0,0


In [9]:
df.shape

(221, 9)

In [10]:
import logging

logger = logging.getLogger(__name__)

logging.basicConfig(
    filename="etl_project_log.txt",
    level=logging.INFO,
    format='%(asctime)s, %(message)s',
    datefmt='%Y-%B-%d-%H-%M-%S'
)

logger.info("test logging")

In [11]:
import sqlite3

# DF을 .json으로 저장
df.to_json("Countries_by_GDP.json", orient='records')

with sqlite3.connect('World_Economies.db') as conn:

    # 이미 테이블이 존재할 시 -> 기존 데이터 새 테이블에 저장? or 효율적으로 .json 파일 등으로 저장? -> 이름은 어떻게 처리?
    df.to_sql('Countries_by_GDP', conn, if_exists='replace', index=True)


In [12]:
sql1 = 'SELECT Country, GDP_USD_billion FROM Countries_by_GDP \
    WHERE GDP_USD_billion >= 100'

with sqlite3.connect('World_Economies.db') as conn:
    cursor = conn.cursor()

    cursor.execute(sql1)
    rows = cursor.fetchall()

    for row in rows:
        print(row)
    

('United States', 30507.22)
('China', 19231.71)
('Germany', 4744.8)
('India', 4187.02)
('Japan', 4186.43)
('United Kingdom', 3839.18)
('France', 3211.29)
('Italy', 2422.86)
('Canada', 2225.34)
('Brazil', 2125.96)
('Russia', 2076.4)
('Spain', 1799.51)
('South Korea', 1790.32)
('Australia', 1771.68)
('Mexico', 1692.64)
('Turkey', 1437.41)
('Indonesia', 1429.74)
('Netherlands', 1272.01)
('Saudi Arabia', 1083.75)
('Poland', 979.96)
('Switzerland', 947.12)
('Taiwan', 804.89)
('Belgium', 684.86)
('Argentina', 683.53)
('Sweden', 620.3)
('Ireland', 598.84)
('Israel', 583.36)
('Singapore', 564.77)
('United Arab Emirates', 548.6)
('Thailand', 546.22)
('Austria', 534.3)
('Norway', 504.28)
('Philippines', 497.5)
('Vietnam', 490.97)
('Bangladesh', 467.22)
('Denmark', 449.94)
('Malaysia', 444.98)
('Colombia', 427.77)
('Hong Kong', 424.0)
('South Africa', 410.34)
('Romania', 403.39)
('Czech Republic', 360.24)
('Egypt', 347.34)
('Chile', 343.82)
('Iran', 341.01)
('Portugal', 321.44)
('Finland', 303.94

In [13]:
def sql2(region):
    return f'SELECT "{region.upper()}", ROUND(AVG(GDP_USD_billion), 2)\
        FROM (SELECT Country, GDP_USD_billion FROM Countries_by_GDP \
                WHERE {region} = 1 \
                ORDER BY GDP_USD_billion DESC \
                LIMIT 5)'

with sqlite3.connect('World_Economies.db') as conn:
    cursor = conn.cursor()

    for region in regions_str:
        cursor.execute(sql2(region))
        rows = cursor.fetchall()

        for row in rows:
            print(row)
    

('AFRICAN', 276.13)
('ARAB', 501.32)
('AP', 6233.43)
('LATIN', 1054.74)
('NA', 6934.84)
('OCEANIAN', 412.27)
('EUROPE', 3258.91)


In [14]:
pd.read_json('Countries_by_GDP.json')

,Country,GDP_USD_billion,african,arab,ap,latin,na,oceanian,europe
0,United States,30507.22,0,0,0,0,1,0,0
1,China,19231.71,0,0,1,0,0,0,0
2,Germany,4744.80,0,0,0,0,0,0,1
3,India,4187.02,0,0,1,0,0,0,0
4,Japan,4186.43,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
216,American Samoa,NaN,0,0,0,0,0,0,0
217,Saint Martin,NaN,0,0,0,0,0,0,0
218,Anguilla,NaN,0,0,0,0,0,0,0
219,Cook Islands,NaN,0,0,0,0,0,0,0


### 팀 활동
- wikipeida 페이지가 아닌, IMF 홈페이지에서 직접 데이터를 가져오는 방법은 없을까요? 어떻게 하면 될까요?
  - IMF의 API를 사용
  - IMF 홈페이지에 게시된 데이터를 직접 다운로드해 사용 or 브라우저 컨트롤 툴을 이용해 다운로드 하도록 프로그램 설계
-  과거의 데이터는 어떻게 되어야 할까요? 과거의 데이터를 조회하는 게 필요하다면 ETL 프로세스를 어떻게 변경해야 할까요?
  - 데이터를 저장할 때 날짜 컬럼과 함께 저장
  - 테이블 정보는 갱신시키고 용량이 적고 보관이 용이한 .json 파일을 원하는 날짜로 불러와 사용할 수 있게끔 파일명에 날짜를 명시에 저장

### 과제 수행 시 마주한 어려움
  - 리전의 정보가 중복되는 국가들이 존재했음
  -> 리전 컬럼 대신 각 리전 이름의 컬럼으로 원핫인코딩함
  - 위키피디아를 통해 가져온 리전에 해당하는 국가들의 표기가 전체 국가들과 다른 국가들이 존재했음
  -> 과제의 목적인 상위 5개의 국가 이름은 동일하게 표기되어 있어 따로 처리하지 않고 수행